# Specification Search

### Luc Anselin

### (revised 10/18/2024)


## Preliminaries

This notebook reviews five different strategies for a spatial regression model specification search, following the principles outlined in Anselin, Serenini and Amaral (2024) *Spatial Econometric Model Specification Search: Another Look* (DOI: 10.13140/RG.2.2.10650.86721). Three strategies are forward search strategies, from specific to general (STGE), and two are backward search strategies, from general to specific (GETS). They are based on a range of Lagrange Multiplier test statistics as well as simple asymptotic t-tests on the significance of the spatial parameters.

### Prerequisites

Familiarity with OLS estimation in *spreg* is assumed, as well as basics of *numpy*, *pandas*, *geopandas*, and *libpysal*. In addition, it is assumed that the **chicagoSDOH** PySAL sample data set has been installed.

### Modules Needed

The main module for spatial regression in PySAL is *spreg*. In addition *libpysal* is needed for data import and spatial weights manipulation, and *geopandas* for data input from a shape file. This notebook is based on version 1.8 of *spreg*. 

As before, only the needed functions from *libpysal* are imported, i.e., `libpysal.io.open` as `open`, `libpysal.examples.get_path` as `get_path`, and `libpysal.weights` as `weights`. The `OLS`, `GM_Lag` and `spsearch` modules are imported from `spreg`.

Some additional imports are included to avoid excessive warning messages. With later versions of PySAL, these may not be needed. As before, the `set_printoptions` is used for *numpy* 2.0 and later.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd
import geopandas as gpd
from libpysal.io import open
from libpysal.examples import get_path
import libpysal.weights as weights
from spreg import OLS, GM_Lag, spsearch
np.set_printoptions(legacy="1.25")

### Functionality Used

- from geopandas:
  - read_file
  
- from libpysal:
  - examples.get_path
  - io.open
  - weights.transform
  
- from spreg:
  - OLS
  - GM_Lag
  - spsearch.stge_classic
  - spsearch.stge_kb
  - spsearch.stge_pre
  - spsearch.gets_gns
  - spsearch.gets_sdm


### Data, Weights and Variables

As in the previous notebooks, all data sets, weights files and variables are specified at the top, so that they can be easily changed to other examples.

Data sets and weights are from the **chicagoSDOH** sample data set:

- **Chi-SDOH.shp,shx,dbf,prj**: socio-economic indicators of health for 2014 in 791 Chicago tracts
- **Chi-SDOH_q.gal**: queen contiguity spatial weights created with *GeoDa*

The weights are used in row-standardized form.

The initial model specification has **YPLL_rate** (an index measuring premature mortality, i.e., higher values are worse health outcomes) as the dependent variable, and **HIS_ct** (economic hardship index), **Blk14P** (percent Black population), and **Hisp14P** 
(percent Hispanic population) as the explanatory variables. These are specified in the **y_name** etc. variables, 

The various initializations are carried out in two steps:

- first, all file names and variable names are defined
- second, the files are read and variable vectors/matrices constructed

The first step allows for customization to other examples, the second step is agnostic to the actual files and variables that were specified. To keep the code simple, there are no error checks for missing files or mismatches in the variable names.

#### Specify file and variable names

In [2]:
infileshp = get_path("Chi-SDOH.shp")            # input shape file with data
infileq = get_path("Chi-SDOH_q.gal")            # queen contiguity weights from GeoDa
y_name = 'YPLL_rate'
x_names = ['Blk14P','Hisp14P','HIS_ct']
ds_name = 'Chi-SDOH'
w_name = 'Chi-SDOH_q'

#### Read files and extract variables

In [3]:
dfs = gpd.read_file(infileshp)
wq =  open(infileq).read()    # queen contiguity weights
wq.transform = 'r'    # row-transform the weights
y = dfs[y_name]
x = dfs[x_names]

### Baseline regression

For easy reference, the results of the base OLS regression are listed as well, without any diagnostics.

In [5]:
reg1 = OLS(y,x,name_ds=ds_name,
           nonspat_diag=False)
print(reg1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :        None
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           4
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         787
R-squared           :      0.6330
Adjusted R-squared  :      0.6316

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     t-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       938.33431       217.72938         4.30964         0.00002
              Blk14P        42.09396         3.49108        12.05757         0.00000
             Hisp14P       -14.55534         4.43781        

## STGE-Classic

The first forward strategy, the Classic Specific-to-General (STGE-Classic), is based on the application of the Lagrange Multiplier tests for
the lag and error alternatives and their robust counterparts.

The logic is a slight adjustment to the flow chart presented in Figure 5.1 on p. 110 of Anselin and Rey (2014). It is augmented with the AK test for
remaining spatial autocorrelation in a spatial lag model. 

The specification search is based on the results of a non-spatial
OLS regression and consists of the following steps:
- compute $LM_{\rho}$ and $LM_{\lambda}$: if none are significant, stick with the classic non-spatial regression (for simplicity, hereafter referred to as OLS); if one is significant and the other is not, select that alternative (Lag
or Error); if both are significant, move to the next step
- compute the robust versions $LM_{\rho}^*$ and $LM_{\lambda}^*$: if one is significant and the
other is not, go with that alternative; if both are significant, move to the next step
- estimate a spatial lag model (i.e., including $Wy$) by means of Spatial 2SLS and compute the AK test for remaining error autocorrelation: if the latter is significant, then the alternative is SAR-Error, otherwise it is Lag.

The last step is an addition to the original flow chart. As it turns out, in empirical practice, the situation where both robust tests remain significant occurs quite frequently (especially with larger data sets), which
leaves the orginal specification search unresolved. The AK test is preferred over the joint $LM_{\rho\lambda}$ test since the latter tends to over-reject the null, even when only one spatial parameter is significant.

Importantly, the STGE-Classic logic does not consider the spatial Durbin model as an alternative, and thus
it cannot detect SLX, SDM, SLX-Error or GNS.

This strategy is invoked as `spsearch.stge_classic`. The return value is a tuple with the final model and a regression object for the final model. Three required arguments are `y`, `x` and `w`. The other general optional arguments are the same as for a standard regression, e.g., `robust`, `w_lags` and `sig2n_k`.

Three important special optional arguments are `p_value`, set to 0.01 as the default, `finmod`, a flag for the estimation of the final model (default `True`), and `mprint`, a flag for listing the output summary of the final model (default `True`). With the default values for these arguments, the call yields the final model selected and the summary output.

In [7]:
mod, finreg = spsearch.stge_classic(y,x,w=wq,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-Classic: LAG
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
      

The model selected is LAG, although the spatial autoregressive coefficient is only marginally significant.

### Steps in the decision tree

To illustrate the logic of this approach, each step in the process is illustrated, starting with the OLS regression with spatial diagnostics.

In [8]:
reg2 = OLS(y,x,w=wq,spat_diag=True,
           name_ds=ds_name,name_w=w_name)
print(reg2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           4
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         787
R-squared           :      0.6330
Adjusted R-squared  :      0.6316
Sum squared residual: 2.89733e+09                F-statistic           :    452.5271
Sigma-square        : 3681490.057                Prob(F-statistic)     :  8.593e-171
S.E. of regression  :    1918.721                Log likelihood        :   -7099.872
Sigma-square ML     : 3662873.167                Akaike info criterion :   14207.744
S.E of regression ML:   1913.8634                Schwarz criterion     :   14226.437

------------------------------------------------------------

Since the `p_value` was set to 0.01, the initial LM-tests point to a Lag alternative, since LM-Lag has p=0.0071, but LM-Error only achieves p=0.0169. Therefore, the next step is to estimate a spatial lag model and carry out the AK test.

In [9]:
reg3 = GM_Lag(y,x,w=wq,spat_diag=True,w_lags=2,
              name_ds=ds_name,name_w=w_name)
print(reg3.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
             Hisp14P       -13.23575      

In this example, the AK-test is not significant, hence the final model is the LAG specification.

### P-value

To assess the sensitivity of the result to the p-value used as the decision criterion, `p_value` is set to 0.05.

In [10]:
mod, finreg = spsearch.stge_classic(y,x,w=wq,name_ds=ds_name,
                      p_value=0.05)

Model selected by STGE-Classic: LAG_Nr
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :       False
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
   

Again, the end result is a LAG specification, but it is achieved via the LAG_Nr route. This is when both initial LM tests are significant (LM-Error becomes significant for p=0.05), but neither of the robust ones are. Then the decision tree takes the least insignificant of the two, in this case LAG.

## STGE-KB

In Koley and Bera (2024), robust versions are derived for the LM tests for $\rho$ and $\gamma$ in the spatial Durbin model. Again, the point of departure is
an OLS regression of the classic non-spatial specification. They recommend the following strategy:
- compute the joint $LM_{\rho\gamma}$ test for SDM: if the test is not significant, stick with the
       non-spatial specification; if the null is rejected, move to the next step
- compute the robust $LM_{\rho}^*$ and $LM_{\gamma}^*$ tests: if one is significant and the other is not, select
that alternative; if both are significant, select SDM.

Given the two out of three problem, these LM tests do not consider the SAR-Error model in the decision tree. As a result, the strategy cannot identify the Error, SAR-Error, SLX-Error and GNS alternatives.

This second strategy is invoked with `spsearch.stge_kb`. The required arguments and options are the same as for `stge_classic`.

In [11]:
mod, finreg = spsearch.stge_kb(y,x,w=wq,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-KB: SLX
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           7
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         784
R-squared           :      0.6420
Adjusted R-squared  :      0.6392
Sum squared residual: 2.82674e+09                F-statistic           :    234.2927
Sigma-square        : 3605541.065                Prob(F-statistic)     :  4.384e-171
S.E. of regression  :    1898.826                Log likelihood        :   -7090.117
Sigma-square ML     : 3573633.622                Akaike info criterion :   14194.234
S.E of regression ML:   1890.4057                Schwar

From the OLS diagnostics for Spatial Durbin in **reg2**, the joint LM test for SDM is highly significant. The next step is then to check the robust LM tests for WX and Lag. Of these, only the test for WX is significant at p=0.01. As a result, the model selected is the SLX model.

### P-value

From the regression diagnostics in **reg2**, it follows that the decision will be different if a more tolerant p-value of 0.05 is used. Since both robust tests are significant in that case, the chosen model is the Spatial Durbin model. However, in this case, the spatial autoregressive coefficient is not significant, and neither are all but one of the WX coefficients.

In [12]:
mod, finreg = spsearch.stge_kb(y,x,w=wq,
                        p_value = 0.05,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-KB: SDM
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           8
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         783
Pseudo R-squared    :      0.6232
Spatial Pseudo R-squared:  0.6405

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       -26.16491       339.30742        -0.07711         0.93853
              Blk14P        56.7

## STGE-Pre

Neither the STGE-Classic nor the STGE-KB strategies consider the full array of spatial
model alternatives. A third forward strategy accomplishes this by means of a pre-test. The starting point
is again an OLS estimation of the classic non-spatial regression model. Based on the Koley-Bera $LM_{\gamma}$
and $LM_{\gamma}^*$ tests on the SLX parameters, a decision is made whether or not to include
$WX$ in the estimation. If the null is not rejected, the initial estimates are used and the STGE-Classic strategy is followed. If the null is rejected, the model is re-estimated as an SLX specification and the
STGE-Classic strategy is applied to these results (an SLX specification is also estimated by OLS, so the STGE-Classic strategy is appropriate).

This yields the full array of alternatives:
- from the original regression: OLS, Lag, Error and SAR-Error
- from the SLX estimation: SLX, SDM, SLX-Error and GNS.


The STGE-Pre strategy is invoked with `spsearch.stge_pre`. The required arguments and options are again the same.

In [13]:
mod, finreg = spsearch.stge_pre(y,x,w=wq,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-Pre: SLX
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           7
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         784
R-squared           :      0.6420
Adjusted R-squared  :      0.6392
Sum squared residual: 2.82674e+09                F-statistic           :    234.2927
Sigma-square        : 3605541.065                Prob(F-statistic)     :  4.384e-171
S.E. of regression  :    1898.826                Log likelihood        :   -7090.117
Sigma-square ML     : 3573633.622                Akaike info criterion :   14194.234
S.E of regression ML:   1890.4057                Schwa

Starting with the LM results for Spatial Durbin in the OLS regression (**reg2**), both the LM test for WX and its robust form are significant at p=0.01. Therefore, the next step is the SLX regression. The diagnostics for the SLX regression obtained for p=0.01 with STGE-KB show that none of the LM tests are significant at p=0.01. Therefore, the selected model is SLX.

### P-value

With a p-value of 0.05, the decision path changes. Since both LM tests for WX were significant at p=0.01, they obviously will also be significant at p=0.05, so the next step is again the SLX estimation. However, now both LM-Lag and LM-Error tests are significant at p=0.05 in the SLX model. Neither of the robust forms are, but the LAG test is larger than the Error test, hence LAG is selected as the alternative (the LAG_Nr path in the classic strategy). Since this pertains to an SLX specification, the result is a Spatial Durbin model.

In [14]:
mod, finreg = spsearch.stge_pre(y,x,w=wq,
                        p_value = 0.05,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-Pre: SDM
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           8
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         783
Pseudo R-squared    :      0.6232
Spatial Pseudo R-squared:  0.6405

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       -26.16491       339.30742        -0.07711         0.93853
              Blk14P        56.

## GETS-GNS

The point of departure in the GETS-GNS strategy is the estimation of a full GNS model. Next, the significance of the individual coefficients is assessed. When one or more coefficients are not significant,
the simplified model (with zero restrictions imposed) is estimated and the process is repeated. All estimations
use IV-GMM methods.

One additional step is carried out when the selected model turns out to be SDM. In that case, the common
factor hypothesis is tested. If the latter is not rejected, then the selected model is the Error model rather than
the unrestricted SDM.

This strategy is included in `spsearch` only for the sake of completeness. As the simulation results in Anselin, Serenini and Amaral (2024) show, it is generally unreliable and very often yields parameter estimates outside the acceptable range. In `spreg`, the parameter space is enforced with the `hard_bound = True` option. This option cannot be relaxed here since a model that yields unacceptable parameter estimates should not be considered a valid specification.

The GETS-GNS strategy is invoked with `spsearch.gets_gns`. The required arguments and options are again the same.

In [15]:
mod, finreg = spsearch.gets_gns(y,x,w=wq,
                        name_ds=ds_name,name_w=w_name)

Exception: GNS parameters out of bounds


In this example, the GNS model is not a valid point of departure. As mentioned, this strategy is generally unreliable and not recommended for empirical practice.

## GETS-SDM

The GETS-SDM strategy is a hybrid that contains aspects of both General-to-Specific (GETS) and Specific-to-General (STGE) approaches. In contrast to GETS-GNS, the point of
departure is not the GNS specification, but instead an SDM model is used. If both $\rho$ and $\gamma$ coefficients are significant, then
a common factor test is carried to assess whether the alternative may be an Error model. If the common factor null hypothesis cannot
be rejected, then the model is Error. In the other case, the SDM specification is kept and
an AK test is carried out on remaining error autocorrelation. If the
null is rejected, then the alternative is GNS, if not, then it remains SDM.

If neither of the coefficients in the SDM model are significant, then the LM-Error tests are used in an OLS regression
of the model without $Wy$ and $WX$ terms, allowing both OLS and Error alternatives. If $\rho$ is significant and $\gamma$ is not,
then an AK test is carried out, yielding either SAR-Error or Lag as the alternatives. In the reverse case, with $\gamma$ significant
and $\rho$ not, then the LM Error tests are used to choose between the SLX and SLX-Error alternatives.

This approach allows the full array of spatial alternatives to be considered.

The GETS-SDM strategy is invoked with `spsearch.gets_sdm`. The required arguments and options are again the same.

In [16]:
mod, finreg = spsearch.gets_sdm(y,x,w=wq,
                        name_ds=ds_name,name_w=w_name)

Model selected by GETS-SDM: LAG
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
          

The point of departure is the SDM model, which is shown in the output of the STGE-Pre strategy. The next step is based on the least significant spatial parameter. If that is the spatial autoregressive coefficient, then the next step is SLX. However, in the current example, two of the WX parameters are less significant than the spatial autoregressive coefficient, and in that case the next step is the LAG model. The motivation is that the consequences of ignoring a lag term are more severe than ignoring an SLX specification. The AK test in the lag model does not reveal remaining autocorrelation, so the recommendation is the LAG specification. The results are not affected by setting p=0.05.

## Overall Assessment

The results of the simulation results in Anselin, Serenini and Amaral (2024) point to the crucial role played by the $WX$ terms. When only classic models are considered (specifically,
no spatial Durbin), the augmented STGE-Classic strategy is superior to all and provides reliable guidance as to the proper alternative. 
In contrast, when $WX$ is part of the true DGP, this is no longer the case. Both STGE-Classic, which does not consider SDM as a DGP, and 
STGE-KB, which does, but ignores spatial errors, fall short. Of the three strategies that take into account SDM, STGE-Pre is clearly superior. In addition, it also
performs well in the classic setting, although not quite as well as STGE-Classic.

GETS-SDM
does well in some settings, but not reliably. GETS-GNS yields unreliable results and should be avoided as a strategy. 

While these search strategies provide an easy way to asses which spatial model may be relevant in a given empirical situation, nothing is a replacement of an explicit and careful model specification based on substantive considerations. Also, other misspecifications, such as endogeneity, heteroskedasticity and nonlinearity may affect the indications provided by the test diagnostics.

## Practice

At this point, it would most effective if you could continue with your baseline regression, assess which of the spatial alternatives may be most relevant and compare the recommendations given by the different strategies. Also, consider changing your base model specification and investigate the sensitivity of the search recommendations.